## Account Data

In [0]:
from pyspark.sql.functions import *

In [0]:
Account_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://broze@sabankingkd.dfs.core.windows.net/account/account.csv")


In [0]:
from pyspark.sql.functions import *
# Duplicate check
duplicate_check = Account_df.groupBy("account_id").count().filter("count > 1")
duplicate_check.show()

In [0]:
Final_Account_df = Account_df.select("account_id", "district_id", "frequency", col("parseddate").alias("issuance_date"))
display(Final_Account_df)

## Card Data

In [0]:
Card_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://broze@sabankingkd.dfs.core.windows.net/card/card.csv")
Card_final_df = Card_df.select("card_id", "disp_id", "type", col("fulldate").alias("card_date"))
Card_final_df.show()

In [0]:
client_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://broze@sabankingkd.dfs.core.windows.net/client/client.csv")
display(client_df)

In [0]:
district_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://broze@sabankingkd.dfs.core.windows.net/district/district.csv")
display(district_df)

In [0]:
loan_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://broze@sabankingkd.dfs.core.windows.net/loan/loan.csv")
display(loan_df)

In [0]:
order_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://broze@sabankingkd.dfs.core.windows.net/order/order.csv")
display(order_df)

In [0]:
position_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://broze@sabankingkd.dfs.core.windows.net/position/position.csv")
display(position_df)

In [0]:
transactions_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://broze@sabankingkd.dfs.core.windows.net/transactions/transactions2013.csv")
display(transactions_df)